In [2]:
import os

from   astropy.table import Table
from   desimodel.footprint import tiles2pix


root  = '/global/cscratch1/sd/mjwilson' + '/altmtl'


---- Get SV3 tiles and assign rosette ----
                                                                                                                                                                                                 
sv3_tiles = /global/cfs/cdirs/desi/survey/ops/surveyops/trunk/ops/tiles-sv3.ecsv                                                                                                                     
sv3_tiles = sv3_tiles[sv3_tiles['PROGRAM'] == 'BRIGHT']                                                                                                                                              
sv3_tiles                                                                                                                                                                                                                                                                                                                                                                                               
                                                                                             
ros       = tiles2rosette(sv3_tiles['TILEID'])                                                                                                                                                       
----  Limit tiles to rosettes: G12: [1,2]; G15: [8,9,10, 17] ----


                                                                                                                                                                                              
----  For each tile remaining, calculating the pix list at the desitarget nside  ---- 
      pixarea2nside(7.) = 32? TBC.
 
TILEID    RA     DEC   OBSCONDITIONS IN_DESI PROGRAM                                                                                                                                                 
int32  float64 float64     int32      int16   bytes6                                                                                                                                                
------ ------- ------- ------------- ------- -------                                                                                                                                                
    37 179.481  -0.018            15       1     SV3                                                                                                                                                                                                                                                                                                                                                     
dat = Table.read('000037-tiles.fits')                                                                                                                                                                                                                                                                                                                                                                    
tiles2pix(32, tiles=dat, radius=None, per_tile=True, fact=2**7)                                                                                                                                     



For every BGS Bright galaxy, assign a hp at the same nside.   By doing so, 
get a target list for each tile in the above rosettes.
Limit to minimal columns:
['RA', 'DEC', 'OBSCONDITIONS', 'SV3_DESI_TARGET', 'SV3_BGS_TARGET', 'SV3_MWS_TARGET', 'TARGETID', 'NUMOBS_MORE', 'PRIORITY', 'SUBPRIORITY']
assigning modal values, as before. 
write to 
/global/cscratch1/sd/mjwilson/altmtl/targets/
starting with one tile to check, with tileid 000037.
'''

tpath = root + '/original/000037-targ.fits'
targ  = Table.read(tpath)

for x in sorted(targ.dtype.names):
    print(x)

# Complete list of columns needed by fba_run (could be less still). 
# https://github.com/desihub/desitarget/blob/b3c58c89bbc5e07902154a0f0d890f62d4e29539/py/desitarget/io.py#L53
min_cols = ['RA', 'DEC', 'OBSCONDITIONS', 'SV3_DESI_TARGET', 'SV3_BGS_TARGET', 'SV3_MWS_TARGET', 'TARGETID', 'NUMOBS_MORE', 'PRIORITY', 'SUBPRIORITY']
targ     = targ[min_cols]
targ.pprint()

#targ.write(root + '/000037-targ.fits', format='fits', overwrite=True)

print('Done.')

FileNotFoundError: [Errno 2] No such file or directory: '/global/cscratch1/sd/mjwilson/altmtl/original/000037-targ.fits'

In [31]:
import numpy as np
def tile2rosette(tile):
    if tile < 433:
        return (tile-1)//27
    else:
        if tile >= 433 and tile < 436:
            return 13
        if tile >= 436 and tile < 439:
            return 14
        if tile >= 439 and tile < 442:
            return 15
        if tile >= 442 and tile <=480:
            return (tile-442)//3
            
        if tile > 480:
            return tile//30    
    return 999999 #shouldn't be any more?

sv3_tiles = Table.read('/global/cfs/cdirs/desi/survey/ops/surveyops/trunk/ops/tiles-sv3.ecsv')                                                                                                                
sv3_tiles = sv3_tiles[sv3_tiles['PROGRAM'] == 'BRIGHT']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
                                                                                             
ros = []
for i in range(len(sv3_tiles['TILEID'])):
    ros.append(tile2rosette(sv3_tiles['TILEID'][i]))
ros = np.array(ros)

ros_mask = np.in1d(ros, [1, 2, 8,9,10,17])

sv3_tiles = sv3_tiles[ros_mask]

pixels = tiles2pix(32, tiles=sv3_tiles['TILEID'], radius=None, per_tile=True,fact=2**7) 

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [23]:
sv3_tiles

TILEID,PASS,RA,DEC,PROGRAM,IN_DESI,PRIORITY,STATUS,EBV_MED,DESIGNHA,DONEFRAC,AVAILABLE
int64,int32,float64,float64,str6,bool,float64,str8,float32,float32,float32,bool
39,0,179.599,0.12,BRIGHT,True,1.620e+00,done,0.02297556,0.00,0.9889,True
40,1,179.51,-0.079,BRIGHT,True,1.458e+00,done,0.023208601,0.00,1.0560,True
41,2,179.719,-0.016,BRIGHT,True,1.312e+00,done,0.023272948,0.00,1.1329,True
42,3,179.49,0.049,BRIGHT,True,1.181e+00,done,0.022991067,0.00,1.1621,True
43,4,179.635,-0.115,BRIGHT,True,1.063e+00,done,0.023390751,0.00,0.9846,True
44,5,179.664,0.101,BRIGHT,True,9.566e-01,done,0.023026964,0.00,0.9841,True
45,6,179.534,0.1,BRIGHT,True,7.972e-01,done,0.022917073,0.00,1.0469,True
46,7,179.567,-0.115,BRIGHT,True,7.174e-01,done,0.023307735,0.00,1.0724,True
47,8,179.709,0.051,BRIGHT,True,6.457e-01,done,0.023171157,0.00,1.3754,True
